#### Decision Tree

In [1]:
# Imports

import numpy as np
import math
from scipy import stats
from prepare_data import *
from statistics import *

In [2]:
def binarize_data(data):
    # calculate mean of each column
    data = (data > data.mean(axis=0)).astype(int)
    return data

In [3]:
class Node:
    def __init__(self, value=None):
        self.value = value
        self.right = None
        self.left = None
        
    def getValue(self):
        return self.value

    def setValue(self, value):
        self.value = value

    def getRight(self):
        return self.right

    def setRight(self, right):
        self.right = right

    def getLeft(self):
        return self.left

    def setLeft(self, left):
        self.left = left    
        

In [4]:
class DecisionTree():
    
    def __init__(self):
        self.decisionTree = None
            
    def decision_id_3(self, examples, attributes, default):
        s_train_X = examples[:, :-1]
        train_Y = examples[:, -1]
        if len(s_train_X) == 0:
            return Node(default)
        elif ((train_Y[0] == train_Y).all()):
            return Node(train_Y[0])
        elif (len(attributes) == 1):
            return Node(stats.mode(train_Y).mode[0])
        else:
            best = self.choose_attribute(attributes, examples)
            best_feature = attributes[best]
            decisionTree = Node(best_feature)
            new_attributes = np.delete(attributes, best)
            new_examples = np.delete(examples, best, 1)
            non_spam = new_examples[(examples[:, best] == 0)]
            leftBranch = self.decision_id_3(non_spam, new_attributes, stats.mode(train_Y).mode[0])
            decisionTree.left = leftBranch  
            spam = new_examples[(examples[:, best] == 1)]
            rightBranch = self.decision_id_3(spam, new_attributes, stats.mode(train_Y).mode[0])
            decisionTree.right = rightBranch          
            return decisionTree

    def choose_attribute(self, attributes, examples):
        attributes_entropies = []
        eps = np.finfo(float).eps
        # loop through columns
        for i in range(0, len(attributes)-1):
            # get rows where i is 0
            row_0 = examples[(examples[:, i] == 0)]
            zero_rows = row_0.shape[0]
            
            # get rows where i is 1
            row_1 = examples[(examples[:, i] == 1)]
            one_rows = row_1.shape[0]
            
            # last element in row 0 is 0
            n0 = row_0[(row_0[:, -1] == 0)]
            n0_rows = n0.shape[0]
            
             # Y element in row 0 is 1
            p0 = row_0[(row_0[:, -1] == 1)]
            p0_rows = p0.shape[0]
            
            # Y element in row 1 is 0
            n1 = row_1[(row_1[:, -1] == 0)]
            n1_rows = n1.shape[0]
            
            # Y element in row 1 is 1
            p1 = row_1[(row_1[:, -1] == 1)]
            p1_rows = p1.shape[0]         
    
            e0 = zero_rows/(len(examples))
            e1 = one_rows/(len(examples))
            
            e0_ = self.calculate_entropy(p0_rows, n0_rows, zero_rows)
            e1_ = self.calculate_entropy(p1_rows, n1_rows, one_rows)
                        
            entropy = (e0 * e0_) + (e1 * e1_)            
            attributes_entropies.append(entropy)
        index = attributes_entropies.index(np.min(attributes_entropies))
        return index
            
    def calculate_entropy(self, p, n, num_rows):
        eps = np.finfo(float).eps
        p_ = p/(num_rows+eps)
        n_ = n/(num_rows+eps)
        H = -n_*np.log2(n_ + eps) - p_*np.log2(p_ + eps)
        return H
    
    def traverse_tree(self, root, s_test_Xmat_binarize):
        predictions = []

        for i in range(s_test_Xmat_binarize.shape[0]):
            node = root
            terminate = False
            while not terminate:
                column = node.getValue()
                if s_test_Xmat_binarize[i][column] == 0:
                    node = node.getLeft()
                else:
                    node = node.getRight()
                if (node.getLeft() == None and node.getRight() == None):
                    predictions.append(node.getValue())
                    terminate = True
        return predictions

In [5]:
train_x, test_x, train_y, test_y = split_data()
s_train_x, s_test_x = standardize_data(train_x, test_x, False)

s_train_binarize = np.concatenate((binarize_data(s_train_x), train_y), 1)
s_test_x_binarize = binarize_data(s_test_x)

attributes = np.arange(0, s_train_binarize.shape[1] - 1)

y = stats.mode(s_train_binarize[:, -1]).mode[0]
decisionTree = DecisionTree()
TreeNode = decisionTree.decision_id_3(s_train_binarize, attributes, y)
            
predictions = decisionTree.traverse_tree(TreeNode, s_test_x_binarize)

TP, TN, FP, FN = confusion_matrix(predictions, test_y)      
print("accuracy: ", calc_accuracy(TP, TN, test_y.shape[0]) * 100, "%")
print("precision: ", calc_precision(TP, FP) * 100, "%")
print("recall: ", calc_recall(TP, FN) * 100, "%")
print("f_measure: ", calc_f_measure(TP, FP, FN) * 100, "%")

accuracy:  90.99804305283757 %
precision:  86.37873754152824 %
recall:  90.27777777777779 %
f_measure:  88.28522920203736 %
